In [1]:
#pip install keras
#pip install TensorFlow
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding
from keras.optimizers import SGD

Using TensorFlow backend.


In [2]:
X = np.load("London/X_s.npy")#68106,125
Y_NO2 = np.load("London/Y_NO2_s.npy")
Y_PM10 = np.load("London/Y_PM10_s.npy")
Y_PM25 = np.load("London/Y_PM25_s.npy")

In [3]:
train_X_PM25, test_X_PM25, train_Y_PM25, test_Y_PM25 = train_test_split(X, Y_PM25, test_size=0.2, random_state=11)
print(train_X_PM25.shape, test_X_PM25.shape, train_Y_PM25.shape, test_Y_PM25.shape)

(54412, 125) (13604, 125) (54412,) (13604,)


In [4]:
def sigmoid(x, derivative=False):
    sigm = 1 / (1 + np.exp(-x))
    if derivative:
        return sigm * (1. - sigm)
    return sigm

In [5]:
x_train=sigmoid(train_X_PM25)
y_train=sigmoid(train_Y_PM25)
x_test=sigmoid(test_X_PM25)
y_test=sigmoid(test_Y_PM25)
data_dim=x_train.shape[1]
batch_size = 16

In [6]:
#build the model
model = Sequential()
model.add(Dense(64, input_dim=data_dim, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='adam')


Instructions for updating:
Colocations handled automatically by placer.


In [7]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                8064      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 12,289
Trainable params: 12,289
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
#train the model
model.fit(x_train, y_train, epochs=20, verbose=0)

Instructions for updating:
Use tf.cast instead.


In [9]:
score = model.evaluate(x_test, y_test)
print(score)

13604/13604 [==============================] - 1s 40us/step
0.0033620247341797464


In [10]:
# calculate the symmetric mean absolute percentage error (SMAPE)
def get_score(pred, valid_y_exp):
    return np.mean(np.abs(pred - valid_y_exp) / (pred + valid_y_exp) * 2)

In [11]:
y_pre=model.predict(x_test)
error=get_score(y_pre,y_test)
print(error)

0.021352539911402513
